# 🐳 Docker Development Environment Setup Guide

Complete guide for setting up and troubleshooting the Docker development environment for the Brain-Computer Interface Data Compression Toolkit.

## 📋 Prerequisites

- Docker Desktop or Docker Engine installed
- VS Code with Dev Containers extension
- Git for version control
- 8GB+ RAM (16GB recommended)
- 20GB+ free disk space

## 🚀 Quick Start

### 1. Clone and Setup

```bash
# Clone the repository
git clone https://github.com/hkevin01/brain-computer-compression.git
cd brain-computer-compression

# Check Docker is running
docker --version
docker-compose --version
```

### 2. Start Development Environment

```bash
# Start all services
docker-compose -f docker-compose.dev.yml up -d

# Or start specific services
docker-compose -f docker-compose.dev.yml up -d postgres redis mongodb minio
```

### 3. Open in VS Code

```bash
# Open project in VS Code
code .

# Use Command Palette: "Dev Containers: Reopen in Container"
```

## 🏗️ Service Architecture

Our Docker setup includes:

| Service | Port | Purpose | Container |
|---------|------|---------|----------|
| **Backend** | 8888, 8000, 5000 | Python API, Jupyter Lab | `bci-backend-dev` |
| **Frontend** | 3000, 4173, 8080 | React/Vite dev server | `bci-frontend-dev` |
| **PostgreSQL** | 5432 | Primary database | `bci-postgres-dev` |
| **Redis** | 6379 | Caching, sessions | `bci-redis-dev` |
| **MongoDB** | 27017 | Document storage | `bci-mongodb-dev` |
| **MinIO** | 9000, 9001 | S3-compatible storage | `bci-minio-dev` |

## 🔧 Configuration Files

### Key Files Overview

- **`docker-compose.dev.yml`** - Main development orchestration
- **`docker/dev-backend.Dockerfile`** - Python backend container
- **`docker/dev-frontend.Dockerfile`** - Node.js frontend container
- **`.devcontainer/devcontainer.json`** - VS Code Dev Container config
- **`.dockerignore`** - Files to exclude from Docker builds

In [ ]:
# Check current Docker setup status
echo "=== Docker System Info ==="
docker --version
docker-compose --version
docker system info --format 'table {{.Name}}\t{{.Value}}' | head -10

echo "\n=== Container Status ==="
docker-compose -f docker-compose.dev.yml ps

## 🛠️ Build and Run Commands

### Essential Commands

In [ ]:
# Navigate to project root
cd /workspace  # If running inside container
# cd brain-computer-compression  # If running on host

echo "=== Available Docker Compose Commands ==="
echo "1. Build all services:"
echo "   docker-compose -f docker-compose.dev.yml build"
echo ""
echo "2. Build specific service:"
echo "   docker-compose -f docker-compose.dev.yml build backend"
echo "   docker-compose -f docker-compose.dev.yml build frontend"
echo ""
echo "3. Start all services:"
echo "   docker-compose -f docker-compose.dev.yml up -d"
echo ""
echo "4. Start specific services:"
echo "   docker-compose -f docker-compose.dev.yml up -d postgres redis"
echo ""
echo "5. View logs:"
echo "   docker-compose -f docker-compose.dev.yml logs -f backend"
echo ""
echo "6. Stop all services:"
echo "   docker-compose -f docker-compose.dev.yml down"
echo ""
echo "7. Restart service:"
echo "   docker-compose -f docker-compose.dev.yml restart backend"

## 🔍 Troubleshooting Guide

### Common Issues and Solutions

### Issue 1: Frontend Build Failures

**Symptoms:**
- `npm install` permission errors
- `EACCES: permission denied` messages
- Container fails to build

**Solution:**
The Dockerfile installs global packages as root first, then switches to user:

```dockerfile
# Install global development tools as root first
RUN npm install -g --force \
    yarn \
    pnpm \
    vite \
    # ... other packages

# Then create and switch to user
RUN groupadd -r devuser && useradd -r -g devuser devuser
USER devuser
```

In [ ]:
# Fix frontend build issues
echo "=== Rebuilding Frontend Container ==="
docker-compose -f docker-compose.dev.yml build --no-cache frontend

echo "\n=== Testing Frontend Container ==="
docker-compose -f docker-compose.dev.yml run --rm frontend node --version

### Issue 2: Port Conflicts

**Symptoms:**
- `Port 8888 is already allocated`
- `Address already in use`
- Services fail to start

**Solution:**

In [ ]:
# Check what's using our ports
echo "=== Checking Port Usage ==="
for port in 3000 5432 6379 8000 8888 9000; do
    echo "Port $port:"
    lsof -i :$port || echo "  Available"
    echo ""
done

# Kill conflicting processes (be careful!)
# sudo kill $(lsof -t -i:8888)  # Only if you're sure

### Issue 3: Volume Permission Issues

**Symptoms:**
- Cannot write files in mounted volumes
- Permission denied errors
- Files owned by root instead of user

**Solution:**

In [ ]:
# Fix volume permissions (Linux/Mac)
echo "=== Fixing Volume Permissions ==="
echo "Current user: $(whoami) ($(id))"

# Check current ownership
ls -la src/ | head -5

# Fix ownership if needed (Linux/Mac)
# sudo chown -R $USER:$USER src/ tests/ examples/ docs/

echo "\n=== Container User Info ==="
docker-compose -f docker-compose.dev.yml run --rm backend id

### Issue 4: Module Import Errors

**Symptoms:**
- `ModuleNotFoundError: No module named 'torch'`
- `ImportError: No module named 'bci_compression'`
- Python packages not found

**Solution:**

In [ ]:
# Check Python environment in container
echo "=== Python Environment Check ==="
docker-compose -f docker-compose.dev.yml run --rm backend python --version
docker-compose -f docker-compose.dev.yml run --rm backend pip list | grep -E "torch|numpy|scipy"

echo "\n=== PYTHONPATH Check ==="
docker-compose -f docker-compose.dev.yml run --rm backend python -c "import sys; print('\\n'.join(sys.path))"

echo "\n=== Install Package in Development Mode ==="
docker-compose -f docker-compose.dev.yml run --rm backend pip install -e .

### Issue 5: VS Code Dev Container Problems

**Symptoms:**
- "Dev container failed to start"
- Extensions not loading
- Python interpreter not found

**Solution:**

In [ ]:
# Check VS Code Dev Container setup
echo "=== Dev Container Configuration ==="
echo "1. Check .devcontainer/devcontainer.json exists:"
ls -la .devcontainer/

echo "\n2. Validate docker-compose.dev.yml:"
docker-compose -f docker-compose.dev.yml config --quiet && echo "✅ Valid" || echo "❌ Invalid"

echo "\n3. Test backend service manually:"
docker-compose -f docker-compose.dev.yml run --rm backend python -c "print('Backend service working!')"

## 🧪 Testing the Environment

In [ ]:
# Comprehensive environment test
echo "=== 🧪 Testing Docker Environment ==="

echo "\n1. ✅ Testing Backend Python Environment"
docker-compose -f docker-compose.dev.yml run --rm backend python -c "
import numpy as np
import scipy
import torch
print('✅ NumPy version:', np.__version__)
print('✅ SciPy version:', scipy.__version__)
print('✅ PyTorch version:', torch.__version__)
print('✅ CUDA available:', torch.cuda.is_available())
"

echo "\n2. ✅ Testing BCI Compression Package"
docker-compose -f docker-compose.dev.yml run --rm backend python -c "
import sys
sys.path.insert(0, '/workspace/src')
try:
    import bci_compression
    print('✅ BCI Compression package importable')
except ImportError as e:
    print('❌ BCI Compression import failed:', e)
"

echo "\n3. ✅ Testing Database Connections"
docker-compose -f docker-compose.dev.yml run --rm backend python -c "
import socket
import time

def test_connection(host, port, name):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        if result == 0:
            print(f'✅ {name} connection successful')
        else:
            print(f'❌ {name} connection failed')
    except Exception as e:
        print(f'❌ {name} error: {e}')

test_connection('postgres', 5432, 'PostgreSQL')
test_connection('redis', 6379, 'Redis')
test_connection('mongodb', 27017, 'MongoDB')
test_connection('minio', 9000, 'MinIO')
"

## 🎯 Quick Neural Compression Test

In [ ]:
# Test core functionality
echo "=== 🧠 Testing Neural Compression ==="
docker-compose -f docker-compose.dev.yml run --rm backend python -c "
import sys
sys.path.insert(0, '/workspace/src')
import numpy as np

try:
    from bci_compression.algorithms import create_neural_lz_compressor

    # Generate test data
    neural_data = np.random.randn(32, 1000)  # 32 channels, 1000 samples
    print(f'✅ Test data shape: {neural_data.shape}')

    # Test compression
    compressor = create_neural_lz_compressor('balanced')
    compressed, metadata = compressor.compress(neural_data)

    ratio = metadata.get('overall_compression_ratio', 'unknown')
    print(f'✅ Compression ratio: {ratio}x')
    print('✅ Neural compression working!')

except Exception as e:
    print(f'❌ Neural compression test failed: {e}')
    import traceback
    traceback.print_exc()
"

## 🔄 Development Workflow

### Daily Development

1. **Start Services**
   ```bash
   docker-compose -f docker-compose.dev.yml up -d
   ```

2. **Open in VS Code**
   - Open project folder
   - Command Palette → "Dev Containers: Reopen in Container"

3. **Access Services**
   - Jupyter Lab: http://localhost:8888
   - Frontend: http://localhost:3000
   - Backend API: http://localhost:8000
   - MinIO Console: http://localhost:9001

4. **Run Tests**
   ```bash
   # Inside container or using docker-compose run
   pytest tests/ -v
   make test-quick
   ```

5. **Stop Services**
   ```bash
   docker-compose -f docker-compose.dev.yml down
   ```

## 📊 Monitoring and Logs

In [ ]:
# Monitor services
echo "=== 📊 Service Status and Monitoring ==="

echo "\n1. Container Status:"
docker-compose -f docker-compose.dev.yml ps

echo "\n2. Resource Usage:"
docker stats --no-stream --format "table {{.Container}}\t{{.CPUPerc}}\t{{.MemUsage}}\t{{.NetIO}}"

echo "\n3. Recent Logs (Backend):"
docker-compose -f docker-compose.dev.yml logs --tail=10 backend

echo "\n4. Health Checks:"
docker-compose -f docker-compose.dev.yml ps --format "table {{.Name}}\t{{.Status}}\t{{.Ports}}"

## 🔧 Advanced Configuration

### GPU Support (NVIDIA)

For GPU acceleration, ensure:
1. NVIDIA Docker runtime installed
2. Add to docker-compose.dev.yml:

```yaml
backend:
  # ... other config
  deploy:
    resources:
      reservations:
        devices:
          - driver: nvidia
            count: 1
            capabilities: [gpu]
```

### Custom Environment Variables

Create `.env` file:
```env
POSTGRES_PASSWORD=your_secure_password
JUPYTER_TOKEN=your_jupyter_token
CUDA_VISIBLE_DEVICES=0
```

### Performance Optimization

1. **Use BuildKit for faster builds:**
   ```bash
   export DOCKER_BUILDKIT=1
   ```

2. **Increase Docker memory limit:**
   - Docker Desktop → Settings → Resources → Memory (16GB+)

3. **Use volume caching:**
   ```yaml
   volumes:
     - ./src:/workspace/src:cached
   ```

## 🆘 Emergency Troubleshooting

### Complete Reset

In [ ]:
# Nuclear option: Complete reset
echo "=== 🆘 Complete Docker Reset ==="
echo "WARNING: This will destroy all containers, images, and volumes!"
echo "Press Ctrl+C to cancel, or continue to reset everything."
# sleep 10

echo "\n1. Stop all containers:"
docker-compose -f docker-compose.dev.yml down -v

echo "\n2. Remove project containers:"
docker container prune -f

echo "\n3. Remove project images:"
docker image prune -a -f

echo "\n4. Remove volumes:"
docker volume prune -f

echo "\n5. Rebuild from scratch:"
docker-compose -f docker-compose.dev.yml build --no-cache

echo "\n✅ Reset complete. Try starting services again."

## 📝 Useful Commands Reference

### Docker Compose Commands
```bash
# Build and start
docker-compose -f docker-compose.dev.yml up --build -d

# Start specific service
docker-compose -f docker-compose.dev.yml up -d postgres

# Execute command in running container
docker-compose -f docker-compose.dev.yml exec backend bash

# Run one-time command
docker-compose -f docker-compose.dev.yml run --rm backend python test.py

# View logs
docker-compose -f docker-compose.dev.yml logs -f backend

# Scale services
docker-compose -f docker-compose.dev.yml up -d --scale backend=2
```

### VS Code Dev Container Commands
- `Ctrl+Shift+P` → "Dev Containers: Rebuild Container"
- `Ctrl+Shift+P` → "Dev Containers: Reopen Locally"
- `Ctrl+Shift+P` → "Dev Containers: Show Container Log"

### Debugging Commands
```bash
# Check container resource usage
docker stats

# Inspect container configuration
docker inspect bci-backend-dev

# Check Docker daemon logs
sudo journalctl -u docker.service

# Test network connectivity
docker-compose -f docker-compose.dev.yml exec backend ping postgres
```

## ✅ Verification Checklist

Use this checklist to verify your environment is working:

- [ ] Docker and Docker Compose installed and running
- [ ] All services start without errors (`docker-compose ps`)
- [ ] Backend container can import `bci_compression` package
- [ ] Frontend container builds successfully
- [ ] Database services are healthy (postgres, redis, mongodb)
- [ ] Ports are accessible (8888, 3000, 8000, 5432, etc.)
- [ ] VS Code Dev Container opens successfully
- [ ] Python environment has all required packages
- [ ] Jupyter Lab accessible at http://localhost:8888
- [ ] Neural compression test runs successfully
- [ ] Volume mounts work (can edit files)
- [ ] Tests pass (`pytest tests/` or `make test-quick`)

## 🎉 Success!

If all checks pass, your Docker development environment is ready! You can now:

1. **Develop in VS Code** with full IntelliSense and debugging
2. **Run Jupyter notebooks** for data analysis and experimentation
3. **Test algorithms** with the full neural compression toolkit
4. **Access databases** for data storage and retrieval
5. **Build frontend** with hot-reload development server

Happy coding! 🧠💻🚀